<a href="https://colab.research.google.com/github/Lucia1009/TFG-Plataforma-para-la-prediccion-de-potencia-electrica-generada-en-parques-eolicos/blob/desarrollo/modelos/ST_Santiago.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
#--------------------------------------------- DATOS ANTERIORES Y POSTERIORES --------------------------------------------------------------------#
# Definición de los hiperparámetros INPUT_LENGTH y OUTPUT_LENGTH
INPUT_LENGTH = 144    # Registros de 24 h anteriores a la entrada
OUTPUT_LENGTH = 6    # El modelo va a predecir 60 min a futuro
#-------------------------------------------------------------------------------------------------------------------------------------------------#

In [ ]:
def train_val_test_split(serie, tr_size=0.8, vl_size=0.1, ts_size=0.1 ):
    # Definir número de datos en cada subserie
    N = serie.shape[0]
    Ntrain = int(tr_size*N)  # Número de datos de entrenamiento
    Nval = int(vl_size*N)    # Número de datos de validación
    Ntst = N - Ntrain - Nval # Número de datos de prueba

    # Realizar partición
    train = serie[0:Ntrain]
    val = serie[Ntrain:Ntrain+Nval]
    test = serie[Ntrain+Nval:]

    return train, val, test

train, val, test = train_val_test_split(datos_filtrado)

print(train.shape)
print(val.shape)
print(test.shape)

In [ ]:
def escalar_datos(datos):
  scaler = MinMaxScaler()
  scaled_datos = scaler.fit_transform(datos)

  return  scaled_datos, scaler

def train_val_test_split_scaled(datos):
  train, val, test = train_val_test_split(datos)

  train_scaled, scaler_train = escalar_datos(train)
  val_scaled, scaler_val = scaler.transform(val)
  test_scaled, scaler_test = scaler.transform(test)

  return train_scaled, val_scaled, test_scaled, scaler_train, scaler_val, scaler_test

def get_target(datos):
  y=datos['WF_Power']
  X=datos.drop(columns=['WF_Power'])
  return X, y

In [ ]:
def hacer_y_entrenar_modelo():
    N_UNITS = 128

    modelo = Sequential()
    modelo.add(LSTM(N_UNITS, input_shape=(INPUT_LENGTH, X_train_s.shape[1])))
    modelo.add(Dense(y_train_s.shape[1], activation='linear'))
    modelo.compile(optimizer=RMSprop(), loss='mse', metrics=['mae', 'mse', 'R2Score'])
    modelo.summary()

    EPOCHS = 10
    BATCH_SIZE = 512

    # Create a TimeseriesGenerator
    train_generator = TimeseriesGenerator(
        X_train_s, y_train_s, length=INPUT_LENGTH, batch_size=BATCH_SIZE
    )

    # Fit the model using the generator
    model.fit_generator(train_generator,epochs= EPOCHS)

    return modelo

In [ ]:
def predice(scaler_y, X_test_s, y_test_s):
  # test_generator = TimeseriesGenerator(
  #       X_test_s, y_test_s, length=INPUT_LENGTH, batch_size=BATCH_SIZE
  #   )

  # preds = modelo.predict(test_generator)
  preds = modelo.predict(X_test_s)

  preds = scaler_y.inverse_transform(preds)
  # y_test_s = scaler_y.inverse_transform(y_test_s)

  # predicciones = pd.DataFrame(preds, columns=y_test.columns)
  # y_test_df = pd.DataFrame(y_test, columns=y_test.columns)

  return preds

  scores={}
  scores['rmse'] = math.sqrt(mean_squared_error(y_test_s, preds))
  scores['mae'] = mean_absolute_error(y_test_s, preds)
  scores['r2']= r2_score(y_test_s, preds)
  scores['max_error']= max_error(y_test_s, preds)